# Project 2: Student Dropout Classifier
*by Anna Kohnen & Marco Aigner*

This notebook, inspired by [Martins et al.](https://link.springer.com/chapter/10.1007/978-3-030-72657-7_16), demonstrates how to predict students' academic success using different machine-learning algorithms.

---
## Original Research
- Paragraph on the most important points of Martins et al.
    - Comparison of different ml-algorithms
    - Comparison of different upsampling-methods
    - K-Fold-CV
    - Hyperparameter Tuning
    - Results

---
## The Data
The dataset contains data from students from the Polytechnic Institute of Portalegre. It explicitly only includes information known at the time of students' enrollment and comprises features related to their academic path as well as to demographical and social-economic information. There are both numerical and categorical features included in the dataset.

<div class="alert alert-block alert-info"> <b>Caution:</b> The categorical features are encoded as numbers.</div>

---
## The Tasks
The project comprises 6 tasks, listed as follows:

1. To analyze and explore the dataset. To perform data pre-processing and cleansing.
2. To calculate and visualize the correlation of features among each other and with the labels. To discuss an interesting correlation
3. To train (at least) four machine-learning algorithms: One probabilistic, one tree-based, one distance-based and one ensemble method each.
4. To evaluate the models using k-fold cross-validation. To report accuracy, mean standard deviation and a confusion matrix per model. To discuss whether one model is significantly better than the others
5. To pick two favorite models. To discuss which features were most relevant for the students' success. To discuss differences between the two models
6. So export the best performing model as ONNX to compete against other models

---
# Packages
First, import the necessary packages :

In [ ]:
# TODO: Move all imports here
# TODO: Only import the necessary modules to save space
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import numpy as np

# Importing the Data

In [ ]:
FILE_NAME = 'train'
DELIMITER = ',' # TODO: parse all files to use comma separation, then drop this

dataframe = pd.read_csv(f'./data/{FILE_NAME}.csv', delimiter=DELIMITER)
dataframe

# 1. Exploring and Pre-Processing
The dataframe's ``shape`` attribute provides a tuple with the number of rows and columns:

In [ ]:
dataframe.shape

```pd.DataFrame.info()``` details the columns' dtypes:

In [ ]:
dataframe.info()

The output verifies that **1.** there are 36 features and one target variable and **2.** that categorical features are numerically encoded.

---
## 1.1 Pre-Processing

Initialize a new dataframe to save the pre-processed data to:

In [ ]:
student_data = pd.DataFrame()

For convenience and readabiltiy, transform the column names into ```snake_case```

In [ ]:
# create a list of new column names by replacing spaces and '/' with _ and removing the rest
snake_case_columns = dataframe.columns.map(lambda x: x.lower().replace(' ', '_').replace('/','_').replace('(','').replace(')', '').replace('\t', '').replace('\'s','')).to_list()
student_data = dataframe.rename(columns=dict(zip(dataframe.columns, snake_case_columns))) # apply the snake_case column names

Fixe a typo in a column-name:

In [ ]:
student_data.rename(columns={'nacionality':'nationality'}, inplace=True)

Categorical columns (known from the [documentation](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success)) can be parsed to pandas' explicit ```categorical``` dtype:

In [ ]:
# manually create a list of categorical column names
categorical_columns = ['marital_status', 'application_mode', 'application_order', 'course', 'daytime_evening_attendance', 'previous_qualification', 'nationality', 'mother_qualification', 'father_qualification', 'mother_occupation', 'father_occupation', 'displaced', 'educational_special_needs', 'debtor', 'tuition_fees_up_to_date', 'gender', 'scholarship_holder', 'international', 'target']

# assign the categorical dtype to respective columns
student_data[categorical_columns] = student_data[categorical_columns].astype('category')

Numerically encode the targets using a dictionary:

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

student_data['target'] = label_encoder.fit_transform(student_data['target'])

## 1.2 General Exploration

Ensure that no data is missing:

In [ ]:
student_data.isnull().values.any() # True if at least a single entry is missing

Ensure that there are no duplicates:

In [ ]:
student_data.duplicated().values.any() # True if at least a single entry exists twice

Inspect random rows of the data:

In [ ]:
student_data.sample(n=5)

Pandas' ``describe()``-method provides useful statistics on the numerical features. 

Notice, how the features' scales differ:

In [ ]:
student_data.describe()

``describe()`` on categorical columns provides different statistics, such as the number of unique as well as the most frequent variable

In [ ]:
student_data.describe(include=['category'])

## 1.3 Numerical Features: Scaling
Scale all numerical features to have zero mean and unit variance:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector as selector

# new dataframe to store the scaled values
student_data_scaled = student_data.copy()

# select numerical columns by their dtype
numerical_columns = student_data_scaled.select_dtypes(include=['int64', 'float64'])

# initialize a standard scaler with default parameters
standard_scaler = StandardScaler()

# scale numerical columns
scaled = standard_scaler.fit_transform(numerical_columns)

# override scaled columns
student_data_scaled[numerical_columns.columns] = scaled

Notice, how the features are more similar after scaling:

(makes them more comparable for the models)

In [ ]:
numerical_column_names = numerical_columns.columns.to_list()

fig, ax = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
ax = ax.flatten()

sns.boxplot(student_data[numerical_column_names], ax=ax[0])

ax[0].set_xticks(numerical_column_names)
ax[0].set_xticklabels(numerical_column_names, rotation=90)
ax[0].set_ylabel('value')
ax[0].set_title('Before')

sns.boxplot(student_data_scaled[numerical_column_names], ax=ax[1])
ax[1].set_xticks(numerical_column_names)
ax[1].set_xticklabels(numerical_column_names, rotation=90)
ax[1].set_title('After')

plt.suptitle('scaling of numerical features')

plt.show()

## 1.4 Categorical Features: One-Hot-Encoding
One-Hot-Encoding creates one column each for distinct values of numerically encoded categorical features so that the models can not accidentally learn ordinal relationships where there are none.

<div class="alert alert-block alert-info"> <b>Omitted: </b>One-Hot-Encoding has been omitted due to time-restrictions but would have been performed otherwise.</div>

## 1.4 Target Variables: Unbalanced Data

Notice, how the dataset is imbalanced, as the 'Graduate' class makes out 50 % of all the data. Such imbalance can lead models to favor the majority classes to achieve higher accuracies.

The imbalance will be dealt with using upsampling-methods later.

In [ ]:
label_distribution = student_data.value_counts(subset=student_data['target'], normalize=True)

fig, ax = plt.subplots()
plot = sns.barplot(data=label_distribution, ax=ax)
plot.set_xticks(range(0, len(label_distribution)))
plot.set_xticklabels(list(label_encoder.classes_))
plot.bar_label(plot.containers[0], fmt='{0:.2f}')
ax.set_ylim(0, 1)
ax.set_title('class imbalance')

plt.show()

# 2. Correlation Analysis

## 2.1 Features With The Target
Features with a very low correlation with the target likely do not provide that much relevant information and are therefore dropped. This allows to focus on highly correlated features instedad.

In [ ]:
# correlate each feature column with the target column
corr_feature_target = student_data_scaled.corrwith(other=student_data_scaled['target'], axis='index', drop=False, method='spearman')

# print the sorted correlations
corr_feature_target.sort_values(ascending=True)

In general, values closer to 0 indicate a low influence on the target variable, values closer to -1/+1 indicate a higher influence.

Drop all features below with a small correlation below a pre-defined threshhold:

In [ ]:
# drop all features with a correlation below 5 %
CORRELATION_TRESHHOLD= 0.05 

# apply the threshhold on the Series
corr_feature_target_threshhold = corr_feature_target[abs(corr_feature_target) >= CORRELATION_TRESHHOLD]

# print the remaining features
corr_feature_target_threshhold_sorted = corr_feature_target_threshhold.sort_values(ascending=True)

corr_feature_target_threshhold_sorted

Notice how the number of features shrinked from 36 to 21.

Plot the feature's correlation with the target:

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))

ax = sns.barplot(corr_feature_target_threshhold_sorted)
ax.set_xticks(range(0, len(corr_feature_target_threshhold_sorted)))
ax.set_xticklabels(corr_feature_target_threshhold_sorted.index.to_list(), rotation=90)
ax.bar_label(ax.containers[0], fmt='{0:.2f}')
ax.set_title("feature correlations with the target")

plt.show()

The bigger an absolute value, the bigger its influence on the target.

### Discussion
In words this, data indicates for example that a lower age corresponds with higher chances of graduation similarly to how the posession of a scholarship does. This seems to make sense as younger students are known to be more likely to study full-time and as extraordinal grades are a key selection criteron for being awarded a scholarship. Grades and approved curricular units from both semester 1 and 2 seem to have the biggest influence on graduation, while on the other hand it is barely influenced by whether students study during daytime or in the evening.

## 2.2 Features With Each Other
As highly correlated features do not add additional information to the model, aim for features that have a high correlation with the target but a low correlation amongst themselves.

A high correlation between single features demonstrates a linear dependency. Highly correlated features have similar effects on the target and therefore do not necessarily add new information to the models when both present.

Therefore, identify groups of highly correlated features and out of them, pick the one feature with the highest correlation with the target.

First calculate the correlation matrix using only the features with a high correlation with the target

In [ ]:
# only use features highly correlated with the target
df_corr_target = student_data_scaled[corr_feature_target_threshhold_sorted.index]

# create the correlation matrix
correlations_features = df_corr_target.corr()

# round to two decimals
correlations_features = correlations_features.round(2) 

correlations_features

To increase reabability, keep only pairwise correlations above a pre-defined threshhold:

In [ ]:
CORRELATION_TRESHHOLD = 0.4 # change as desired

# remove correlations below the threshhold
correlations_features_filtered = correlations_features[abs(correlations_features) > CORRELATION_TRESHHOLD]

# round to two decimals
correlations_features_filtered = correlations_features_filtered.round(2)

correlations_features_filtered

Plot the correlations

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(15,20))

whole = sns.heatmap(data=correlations_features, annot=True, linewidths=0.1, ax=ax[0])
whole.set_title('all correlations')

filtered = sns.heatmap(data=correlations_features_filtered, annot=True, linewidths=0.1, ax=ax[1])
filtered.set_title('filtered correlations')

fig.tight_layout()

plt.show()

### Discussion

A good example for the logic behind correlations can be seem by the ``age at enrollment`` which is highly correlated with three other features, namely the ``application mode``, ``marital status`` and ``daytime/evening attendance``. Knowing that a lot of people marry between the age of 25 and 30, the positive correlation between the age and marital mode makes sense as it states that younger students are less likely to be married. Furthermore, younger students are more likely to be full-time students, while older students are more likely to study part-time next to their job and possibly family. Thus it makes sense that younger full-time students applied through the general contigent and mostly study during daytime.

Another interesting correlation can be seen in ``debtor`` and ``tuition fees up to date``. One could for example argue, how students that forget to pay their tuition fees could also forget to pay back credits which would lead them into debt.

## 2.3 Feature Selection

Out of a group of pairwise correlated features, pick only the one feature which has the highest correlation with the target variable:

In [ ]:
to_drop = ['marital_status', 'application_mode', 'daytime_evening_attendance', 'previous_qualification', 'previous_qualification_grade', 'debtor', 'curricular_units_1st_sem_enrolled', 'curricular_units_1st_sem_approved', 'curricular_units_1st_sem_grade', 'curricular_units_1st_sem_without_evaluations' ]

Notice how the number of features got reduced from 36 to 11:

In [ ]:
final_dataframe = df_corr_target.drop(columns=to_drop, axis='columns')

final_dataframe

# 3. Model Training

## 3.1 Data Split

In [ ]:
from sklearn.model_selection import train_test_split

X = final_dataframe.drop(columns='target')
y = final_dataframe['target']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

<hr>

<hr>

# Create the models

<hr>

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, StratifiedKFold


# Probabilistic Model
probabilistic_model = GaussianNB()

# Tree-based Model
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# Distance-Based Model
knn = KNeighborsClassifier(n_neighbors=5)

# Ensemble Methods
gbc = GradientBoostingClassifier(n_estimators=100, random_state=42)
xgbc = XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)

models = {
    'Gaussian Naive Bayes': GaussianNB(),

    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),

    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),

    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(random_state=42)
}

# Specify the number of folds for K-fold cross-validation
num_folds = 10

# Create KFold objects
folds = {
    'K-Fold' : KFold(n_splits=num_folds, shuffle=True, random_state=42),
    'Stratified K-Fold' : StratifiedKFold(n_splits=num_folds)
}



In scikit-learn, the KNeighborsClassifier expects the input data to be in the form of a NumPy array or array-like object, not a DataFrame.

This is why we call ".values" on our input data

<hr>

<hr>

# Method to train the model

In [ ]:
def train_model(model, model_name, X_train, y_train):
    try:
        # Train the model
        model.fit(X_train.values, y_train.values)
    except Exception as e:
        print(f"Error while training {model_name}: {e}")

<hr>

<hr>

# Method for cross validation prediction

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def cross_validate_model(model, model_name, X, y, folding, fold_name):
    try:
        # Use cross_val_predict to get predictions for each fold
        y_pred = cross_val_predict(model, X.values, y.values, cv=folding)

        # Calculate confusion matrix
        cm = confusion_matrix(y, y_pred)
        #print(f"Confusion Matrix with {fold_name}:\n{cm}")

        # Calculate accuracy
        accuracy = accuracy_score(y, y_pred)
        print(f"{fold_name} with method cross_val_predict and model: {model_name} - Accuracy: {accuracy:.2f}\n")

        print(f"Classification Report with cross-val-predict for {model_name} and {fold_name} folding:\n")
        print(classification_report(y, y_pred) + "\n")

        return cm

    except Exception as e:
        print(f"Error while cross-validating {model_name}: {e}")
        return None

<hr>

<hr>

# Method to plot confusion Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


def plot_confusion_matrix(model_name, cm_dict):

    # Set the number of rows and columns for the subplots
    num_rows = 1
    num_cols = len(cm_dict)
    
    # Set the figure size
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5))

    # Flatten the axes if only one row
    axes = axes.flatten() if num_rows == 1 else axes

    # Iterate through the dictionary items and create subplots
    for idx, (title, matrix) in enumerate(cm_dict.items()):
        ax = axes[idx]
        sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', ax=ax)
        ax.set_title(title)
        ax.set_xlabel('Predicted')
        ax.set_ylabel('Actual')

    # Set the title for the entire plot
    plt.suptitle(model_name)
    
    # Adjust layout
    plt.tight_layout()
    
    # Show the plot
    plt.show()



<hr>

<hr>

# Methode to get feature importance

In [ ]:
def analyze_feature_importance(model, X):
    feature_importance = model.feature_importances_
    feature_dict = dict(zip(X.columns, feature_importance))

    # Sort the dictionary by values in ascending order
    sorted_dict = dict(sorted(feature_dict.items(), key=lambda item: item[1], reverse=True))
    print(sorted_dict)
    print("\n")

<hr>

<hr>

# Methode that does everything (fit, evaluate (cross-validation), feature importance, plot confusion matrix)

In [ ]:
from sklearn.model_selection import cross_val_score

def train_and_evaluate_model(model, model_name, X, y, folding, fold_name):
    try:
        # Perform K-fold cross-validation and get accuracy scores for each fold
        cv_scores = cross_val_score(model, X.values, y.values, cv=folding, scoring='accuracy')
        print(f"{fold_name} with method cross_val_score and model: {model_name} - Accuracy: {np.mean(cv_scores):.2f} (± {np.std(cv_scores):.2f})\n")

        conf_mat = cross_validate_model(model, model_name, X, y, folding, fold_name)
        
        if conf_mat is not None:
            # Check if the model has feature_importances_ attribute
            if hasattr(model, 'feature_importances_'):
                analyze_feature_importance(model, X)
            else:
                print(f"Feature importance not available for {model_name} with {fold_name} folding." + "\n")

    except Exception as e:
        print(f"Error while processing {model_name}: {e}")
        
    return conf_mat

<hr>

<hr>

# Execute everything for each model

In [ ]:

#TRAIN_FRACTION = 0.75
#train = pd.concat([final_dataframe[final_dataframe['target'] == 0].sample(frac=TRAIN_FRACTION), final_dataframe[final_dataframe['target'] == 1].sample(frac=TRAIN_FRACTION),final_dataframe[final_dataframe['target'] == 2].sample(frac=TRAIN_FRACTION)])
#test = final_dataframe.drop(train.index)
#X_train = test.drop(columns='target')
#y_train = test['target']
#X_test = test.drop(columns='target')
#y_test = test['target']
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC

X = train_data.drop(columns='target')
y = train_data['target']

sm = SMOTENC(random_state=42, categorical_features="auto")
X_res, y_res = sm.fit_resample(X, y)
y_res.shape

X_test = test_data.drop(columns='target')
y_test = test_data['target']
#X = final_dataframe.drop(columns='target')
#y = final_dataframe['target']

X_train,X_test,y_train,y_test = train_test_split(X_res,y_res,test_size=0.2,random_state=42)

X_train = train_data.drop(columns='target')
y_train = train_data['target']
# Loop through your models and call the train_and_evaluate_model method
for model_name, model in models.items():
    cm_dict = {}
    # Train the model
    train_model(model, model_name, X_train, y_train)

    for fold_name, fold in folds.items():
        cm_dict[fold_name] = train_and_evaluate_model(model, model_name, X_res, y_res, fold, fold_name)

    # Plot confusion matrix using seaborn
    plot_confusion_matrix(model_name, cm_dict)


    # Predict for Test-data
    y_pred = model.predict(X_test.values)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Prediction of test set with model: {model_name} - Accuracy: {accuracy:.2f}\n")


PCA, ein random feature nehmen, gini co efficient(rfc), multiple linear regression